# QA of DSL Release v1.23
The purpose of this notebook is to review and test the new features in 1.23

Docs: https://docs.dimensions.ai/dsl/1.23.0-preview/


## Prerequisites

Please install the latest versions of these libraries to run this notebook. 

In [1]:
#
# load common libraries
import pandas as pd
from pandas.io.json import json_normalize

import time
import json
from tqdm.notebook import tqdm as progress

import plotly.express as px
from plotly.offline import plot 

import dimcli
from dimcli.shortcuts import *

dimcli.login(instance="test")
dsl = dimcli.Dsl() 


DimCli v0.6.3 - Succesfully connected to <https://integration.ds-metrics.com> (method: dsl.ini file)


---

## [DSL-345] Add secret .restricted_to field as PF-01

https://uberresearch.atlassian.net/browse/DSL-345

* works ok, with the exception of researchers and organizations

In [25]:
q = """ search {} where pf01 is not empty return {}[id+pf01] limit 1"""

for source in dimcli.G.sources():
    data = dsl.query(q.format(source, source))
    if not data.errors:
        print(getattr(data, source)[0])
                    

Returned Publications: 1 (total = 12682329)
{'id': 'pub.1123466816', 'pf01': 'SN'}
Returned Grants: 1 (total = 586742)
{'id': 'grant.8104257', 'pf01': ['SN']}
Returned Patents: 1 (total = 359097)
{'id': 'EP-2131403-A1', 'pf01': ['SN']}
Returned Clinical_trials: 1 (total = 37145)
{'id': 'NCT00249951', 'pf01': ['SN']}
Returned Policy_documents: 1 (total = 20437)
{'id': 'policy.643831', 'pf01': 'SN'}
Returned Errors: 1
Semantic Error
Semantic errors found:
	Field 'pf01' is not present in source 'researchers'. Available fields are: current_research_org,first_grant_year,first_name,first_publication_year,id,last_grant_year,last_name,last_publication_year,nih_ppid,obsolete,orcid_id,redirect,research_orgs,total_grants,total_publications
Returned Errors: 1
Semantic Error
Semantic errors found:
	Field 'pf01' is not present in source 'organizations'. Available fields are: acronym,city_name,cnrs_ids,country_name,established,external_ids_fundref,hesa_ids,id,ificlaims_ids,isni_ids,latitude,linkout,l

#### Testing that the field is not returned by 'describe'

In [26]:
%dsldocs grants

sources                     field           type  \
0   grants                  abstract         string   
1   grants               active_year        integer   
2   grants              category_bra     categories   
3   grants              category_for     categories   
4   grants              category_hra     categories   
5   grants          category_hrcs_hc     categories   
6   grants         category_hrcs_rac     categories   
7   grants         category_icrp_cso     categories   
8   grants          category_icrp_ct     categories   
9   grants             category_rcdc     categories   
10  grants                  concepts         string   
11  grants             date_inserted           date   
12  grants                  end_date           date   
13  grants                foa_number         string   
14  grants          funder_countries      countries   
15  grants                   funders  organizations   
16  grants               funding_aud          float   
17  grants               funding_cad          float   
18  grants               funding_chf          float   
19  grants          funding_currency         string   
20  grants               funding_eur          float   
21  grants               funding_gbp          float   
22  grants               funding_jpy          float   
23  grants               funding_nzd          float   
24  grants       funding_org_acronym         string   
25  grants          funding_org_city         string   
26  grants          funding_org_name         string   
27  grants               funding_usd          float   
28  grants              grant_number         string   
29  grants                        id         string   
30  grants      investigator_details           json   
31  grants                  language         string   
32  grants            language_title         string   
33  grants                   linkout         string   
34  grants            original_title         string   
35  grants       research_org_cities         cities   
36  grants    research_org_countries      countries   
37  grants  research_org_state_codes         states   
38  grants             research_orgs  organizations   
39  grants               researchers    researchers   
40  grants                start_date           date   
41  grants                start_year        integer   
42  grants                     title         string   

                                          description  is_filter  is_entity  \
0          Abstract or summary from a grant proposal.      False      False   
1                   List of active years for a grant.       True      False   
2   `Broad Research Areas <https://app.dimensions....       True       True   
3   `ANZSRC Fields of Research classification <htt...       True       True   
4   `Health Research Areas <https://app.dimensions...       True       True   
5   `HRCS - Health Categories <https://app.dimensi...       True       True   
6   `HRCS – Research Activity Codes <https://app.d...       True       True   
7   `ICRP Common Scientific Outline <https://app.d...       True       True   
8   `ICRP Cancer Types <https://app.dimensions.ai/...       True       True   
9   `Research, Condition, and Disease Categorizati...       True       True   
10  Concepts describing the main topics of a grant...      False      False   
11  Date when the record was inserted into Dimensi...       True      False   
12                          Date when the grant ends.       True      False   
13  The funding opportunity announcement (FOA) num...       True      False   
14  The country linked to the organisation funding...       True       True   
15  The organisation funding the grant. This is no...       True       True   
16                     Funding amount awarded in AUD.       True      False   
17                     Funding amount awarded in CAD.       True      False   
18                     Funding amount awarded in CHF.       True      False   
19

#### verifying that 'all' does not return the secret field.. 

In [30]:
q = """ search {} where pf01 is not empty return {}[all] limit 1"""

for source in dimcli.G.sources() + ['datasets']:
    data = dsl.query(q.format(source, source))
    if not data.errors:
        record = getattr(data, source)[0]
        print("=== pf01 in fields?:", "pf01" in [x for x in record.keys()])
                    

Returned Publications: 1 (total = 12682329)
WARNINGS [10]
Field 'terms' is deprecated in favor of concepts. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'FOR' is deprecated in favor of category_for. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'open_access' is deprecated in favor of open_access_categories. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'FOR_first' is deprecated in favor of category_for. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'references' is deprecated in favor of reference_ids. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'RCDC' is deprecated in favor of category_rcdc. Please refer to https://docs.dimensions.ai/

IndexError: list index out of range

## DSL-343 Deprecate grant.title_language in favor of grant.language_title 

https://uberresearch.atlassian.net/browse/DSL-343

In [22]:
%dsldf search grants where title_language is not empty return grants[id+title_language] limit 10

Returned Grants: 10 (total = 5037267)
WARNINGS [1]
Field 'title_language' is deprecated in favor of language_title. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


id title_language
0  grant.8690978             en
1  grant.8715161             en
2  grant.8688630             en
3  grant.8688908             en
4  grant.8689245             en
5  grant.8689399             en
6  grant.8690522             en
7  grant.8690709             en
8  grant.8690632             en
9  grant.8689727             en

In [23]:
%dsldf search grants where language_title is not empty return grants[id+language_title] limit 10

Returned Grants: 10 (total = 5037267)


id language_title
0  grant.8690978             en
1  grant.8715161             en
2  grant.8688630             en
3  grant.8688908             en
4  grant.8689245             en
5  grant.8689399             en
6  grant.8690522             en
7  grant.8690709             en
8  grant.8690632             en
9  grant.8689727             en

## DSL-330 grants.resulting_pub_ids

https://uberresearch.atlassian.net/browse/DSL-330

In [21]:
%dsldf search grants where resulting_publication_ids is not empty return grants

Returned Grants: 20 (total = 1381805)
WARNINGS [1]
Field 'resulting_publication_ids' is deprecated. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


active_year    end_date  \
0                     [2020, 2021, 2022]  2022-08-31   
1                     [2020, 2021, 2022]  2022-12-31   
2         [2020, 2021, 2022, 2023, 2024]  2024-12-31   
3                     [2020, 2021, 2022]  2022-12-31   
4                                 [2020]         NaN   
5                                 [2020]         NaN   
6               [2019, 2020, 2021, 2022]  2022-12-14   
7               [2019, 2020, 2021, 2022]  2022-12-14   
8               [2019, 2020, 2021, 2022]  2022-12-14   
9               [2019, 2020, 2021, 2022]  2022-11-30   
10              [2019, 2020, 2021, 2022]  2022-05-31   
11              [2019, 2020, 2021, 2022]  2022-11-30   
12  [2019, 2020, 2021, 2022, 2023, 2024]  2024-11-30   
13              [2019, 2020, 2021, 2022]  2022-11-30   
14              [2019, 2020, 2021, 2022]  2022-11-14   
15  [2019, 2020, 2021, 2022, 2023, 2024]  2024-10-31   
16                    [2019, 2020, 2021]  2021-05-31   
17                    [2019, 2020, 2021]  2021-04-30   
18              [2019, 2020, 2021, 2022]  2022-10-31   
19              [2019, 2020, 2021, 2022]  2022-10-31   

                                              funders  \
0   [{'id': 'grid.270680.b', 'name': 'European Com...   
1   [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
2   [{'id': 'grid.452896.4', 'name': 'European Res...   
3   [{'id': 'grid.453025.5', 'name': 'Office of Nu...   
4   [{'id': 'grid.431143.0', 'name': 'National Hea...   
5   [{'id': 'grid.431143.0', 'name': 'National Hea...   
6   [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
7   [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
8   [{'id': 'grid.452963.f', 'name': 'Office of Bi...   
9   [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
10  [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
11  [{'id': 'grid.453025.5', 'name': 'Office of Nu...   
12  [{'id': 'grid.452896.4', 'name': 'European Res...   
13  [{'id': 'grid.452988.a', 'name': 'Office of Ba...   
14  [{'id': 'grid.453025.5', 'name': 'Office of Nu...   
15  [{'id': 'grid.452896.4', 'name': 'European Res...   
16  [{'id': 'grid.453216.7', 'name': 'Office of Sc...   
17  [{'id': 'grid.453216.7', 'name': 'Office of Sc...   
18  [{'id': 'grid.418100.c', 'name': 'Biotechnolog...   
19  [{'id': 'grid.452988.a', 'name': 'Office of Ba...   

                                     funding_org_name             id language  \
0                                 European Commission  grant.8389274       en   
1                     Office of Basic Energy Sciences  grant.4319742       en   
2                           European Research Council  grant.8104257       en   
3                           Office of Nuclear Physics  grant.4321720       en   
4        National Health and Medical Research Council  grant.7879083       en   
5        National Health and Medical Research Council  grant.7879100       en   
6                     Office of Basic Energy Sciences  grant.6505296       en   
7                     Office of Basic Energy Sciences  grant.4321680       en   
8     Office of Biological and Environmental Research  grant.4319539       en   
9                     Office of Basic Energy Sciences  grant.4321931       en   
10                    Office of Basic Energy Sciences  grant.5495806       en   
11                          Office of Nuclear Physics  grant.4320033       en   
12                          European Research Council  grant.7911731       en   
13                    Office of Basic Energy Sciences  grant.4321103       en   
14                          Office of Nuclear Physics  grant.4320633       en   
15                          European Research Council  grant.7926620       en   
16                                  Office of Science  grant.4321028       en   
17                                  Office of Science  grant.7065373       en   
18  Biotechnology and Biological Sciences Research...  grant.7924204       en   
19                    Office of Ba

## [DSL-342] Deprecate *.category_ua in favor of *.category.uoa

https://uberresearch.atlassian.net/browse/DSL-342

##### publications

In [35]:
%dsldf search publications where category_ua is not empty return category_ua limit 5

Returned Category_ua: 5
WARNINGS [1]
Field 'category_ua' is deprecated in favor of category_uoa. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


count     id                                               name
0  12718005  30012                                    B12 Engineering
1   7582614  30001                              A01 Clinical Medicine
2   5386256  30003  A03 Allied Health Professions, Dentistry, Nurs...
3   3421517  30011               B11 Computer Science and Informatics
4   2653665  30008                                      B08 Chemistry

In [36]:
%dsldf search publications where category_uoa is not empty return category_uoa limit 5

Returned Category_uoa: 5


count     id                                               name
0  12718005  30012                                    B12 Engineering
1   7582614  30001                              A01 Clinical Medicine
2   5386256  30003  A03 Allied Health Professions, Dentistry, Nurs...
3   3421517  30011               B11 Computer Science and Informatics
4   2653665  30008                                      B08 Chemistry

##### datasets

In [37]:
%dsldf search datasets where category_uoa is not empty return category_uoa limit 5

Returned Category_uoa: 5


count     id                                          name
0  224696  30012                               B12 Engineering
1  175022  30001                         A01 Clinical Medicine
2  144379  30007  B07 Earth Systems and Environmental Sciences
3  139504  30008                                 B08 Chemistry
4  132183  30005                       A05 Biological Sciences

In [38]:
%dsldf search datasets where category_ua is not empty return category_ua limit 5

Returned Errors: 1
Semantic Error
Semantic errors found:
	Field 'category_ua' is not present in source 'datasets'. Available fields are: associated_grant_ids,authors,category_bra,category_for,category_hra,category_hrcs_hc,category_hrcs_rac,category_icrp_cso,category_icrp_ct,category_rcdc,category_uoa,date,date_created,date_embargo,date_inserted,date_modified,description,doi,figshare_url,funder_countries,funders,id,journal,keywords,language_desc,language_title,license,pf01,publication_id,reference_ids,repository_id,research_org_cities,research_org_countries,research_org_states,research_orgs,researchers,title,year
	Facet 'category_ua' is not present in source 'datasets'. Available facets are: category_bra,category_for,category_hra,category_hrcs_hc,category_hrcs_rac,category_icrp_cso,category_icrp_ct,category_rcdc,category_uoa,funder_countries,funders,journal,keywords,language_desc,language_title,pf01,repository_id,research_org_cities,research_org_countries,research_org_states,research_or

## [DSL-353] Deprecate clinical_trials.organizations in favor of research_orgs

https://uberresearch.atlassian.net/browse/DSL-353

In [7]:
%dsldf search clinical_trials where organizations = "grid.48336.3a" return clinical_trials limit 1

Returned Clinical_trials: 1 (total = 13656)
WARNINGS [1]
Field 'organizations' is deprecated in favor of research_orgs. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


id                               investigator_details  \
0  NCT00251329  [[Sandra X Franco, MD, Principal Investigator,...   

                                               title  
0  Phase II Neoadjuvant Trial of Docetaxel (Taxot...

In [8]:
%dsldf search clinical_trials where research_orgs = "grid.48336.3a" return clinical_trials limit 1

Returned Clinical_trials: 1 (total = 13656)


id                               investigator_details  \
0  NCT00251329  [[Sandra X Franco, MD, Principal Investigator,...   

                                               title  
0  Phase II Neoadjuvant Trial of Docetaxel (Taxot...

In [10]:
%dsldf search clinical_trials return organizations limit 1

Returned Organizations: 1


acronym  count   country_name             id                       name
0     NCI  13656  United States  grid.48336.3a  National Cancer Institute

In [9]:
%dsldf search clinical_trials return research_orgs limit 1

Returned Research_orgs: 1


acronym  count   country_name             id                       name
0     NCI  13656  United States  grid.48336.3a  National Cancer Institute

In [11]:
%dsldocs clinical_trials

sources                 field           type  \
0   clinical_trials              abstract         string   
1   clinical_trials               acronym         string   
2   clinical_trials          active_years        integer   
3   clinical_trials  associated_grant_ids         string   
4   clinical_trials           brief_title         string   
5   clinical_trials          category_bra     categories   
6   clinical_trials          category_for     categories   
7   clinical_trials          category_hra     categories   
8   clinical_trials      category_hrcs_hc     categories   
9   clinical_trials     category_hrcs_rac     categories   
10  clinical_trials     category_icrp_cso     categories   
11  clinical_trials      category_icrp_ct     categories   
12  clinical_trials         category_rcdc     categories   
13  clinical_trials            conditions         string   
14  clinical_trials                  date           date   
15  clinical_trials         date_inserted           date   
16  clinical_trials      funder_countries      countries   
17  clinical_trials               funders  organizations   
18  clinical_trials                gender         string   
19  clinical_trials                    id         string   
20  clinical_trials         interventions           json   
21  clinical_trials  investigator_details           json   
22  clinical_trials               linkout         string   
23  clinical_trials                 phase         string   
24  clinical_trials       publication_ids         string   
25  clinical_trials              registry         string   
26  clinical_trials         research_orgs  organizations   
27  clinical_trials           researchers    researchers   
28  clinical_trials                 title         string   

                                          description  is_filter  is_entity  \
0      Abstract or description of the clinical trial.      False      False   
1                      Acronym of the clinical trial.       True      False   
2          List of active years for a clinical trial.       True      False   
3   Dimensions IDs of the grants associated to the...       True      False   
4                  Brief title of the clinical trial.       True      False   
5   `Broad Research Areas <https://app.dimensions....       True       True   
6   `ANZSRC Fields of Research classification <htt...       True       True   
7   `Health Research Areas <https://app.dimensions...       True       True   
8   `HRCS - Health Categories <https://app.dimensi...       True       True   
9   `HRCS – Research Activity Codes <https://app.d...       True       True   
10  `ICRP Common Scientific Outline <https://app.d...       True       True   
11  `ICRP Cancer Types <https://app.dimensions.ai/...       True       True   
12  `Research, Condition, and Disease Categorizati...       True       True   
13  List of medical conditions names, e.g. 'Breast...       True      False   
14                    Start date of a clinical trial.       True      False   
15  Date when the record was inserted into Dimensi...       True      False   
16       The country group the funding organisations.       True       True   
17  GRID funding organisations that are involved w...       True       True   
18  The gender of the clinical trial subjects e.g....       True      False   
19                       Dimensions clinical trial ID       True      False   
20  Structured JSON object containing information ...       True      False   
21  Additional details about investigators, includ...       True      False   
22               Original URL for the clinical trial.      False      False   
23          Phase of the clinical trial, as a string.       True      False   
24  Dimensions IDs of the publications related to ...       True      False   
25  The platform where the clinical trial has been...       True      False   
26  GRID organizations involved, e.g. as sponsors ...       True       True   